In [22]:
import pathway as pw

# This creates a connector that tracks files in a given directory.
data_sources = []
data_sources.append(
    pw.io.fs.read(
        "./files",
        format="binary",
        mode="streaming",
        with_metadata=True,
    )
)


In [46]:
data = [*data_sources]
#print(data_sources)
len(data)
print(type(data[0]).__dict__)

{'__module__': 'pathway.internals.table', '__annotations__': {'_columns': 'dict[str, clmn.Column]', '_schema': 'type[Schema]', '_id_column': 'clmn.IdColumn', '_rowwise_context': 'clmn.RowwiseContext', '_source': 'SetOnceProperty[OutputHandle]'}, '__doc__': "Collection of named columns over identical universes.\n\n    Example:\n\n    >>> import pathway as pw\n    >>> t1 = pw.debug.table_from_markdown('''\n    ... age | owner | pet\n    ... 10  | Alice | dog\n    ... 9   | Bob   | dog\n    ... 8   | Alice | cat\n    ... 7   | Bob   | dog\n    ... ''')\n    >>> isinstance(t1, pw.Table)\n    True\n    ", '_source': <pathway.internals.helpers.SetOnceProperty object at 0x7057b00da6d0>, '__init__': <function Table.__init__ at 0x7057b0137240>, 'id': <property object at 0x7057b0138540>, 'column_names': <function Table.column_names at 0x7057b0163560>, 'keys': <function Table.keys at 0x7057b0163600>, '_get_column': <function Table._get_column at 0x7057b01636a0>, '_ipython_key_completions_': <func

In [21]:
# We now build the VectorStore pipeline

from pathway.xpacks.llm.embedders import LiteLLMEmbedder
from pathway.xpacks.llm.splitters import TokenCountSplitter
from pathway.xpacks.llm.vector_store import VectorStoreClient, VectorStoreServer
import time

PATHWAY_PORT = 8765

# Choose document transformers
text_splitter = TokenCountSplitter()
#embedder = OpenAIEmbedder(api_key=os.environ["OPENAI_API_KEY"])
embedder = LiteLLMEmbedder(model='ollama/all-minilm')

# The `PathwayVectorServer` is a wrapper over `pathway.xpacks.llm.vector_store` to accept LangChain transformers.
# Fell free to fork it to develop bespoke document processing pipelines.
vector_server = VectorStoreServer(
    *data_sources,
    embedder=embedder,
    splitter=text_splitter,
)
vector_server.run_server(host="127.0.0.1", port=PATHWAY_PORT, threaded=True, with_cache=False)
time.sleep(30)  # Workaround for Colab - messages from threads are not visible unless a cell is running

# You can connect to the Pathway+LlamaIndex server using any client - Pathway's, Langchain's or LlamaIndex's!
client = VectorStoreClient(
    host="127.0.0.1",
    port=PATHWAY_PORT+1,
)

client.query("pathway")



TypeError: object of type 'NoneType' has no len()